<a href="https://colab.research.google.com/github/QaziSaim/DeepLearning--DeepNeuralNetwork-/blob/main/Practice_Basic_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip uninstall gensim

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.utils import pad_sequences
import pandas as pd

In [2]:
!unzip /content/twitter_training.csv.zip -d /content/extraction_directory/

Archive:  /content/twitter_training.csv.zip
  inflating: /content/extraction_directory/twitter_training.csv  


In [3]:
df = pd.read_csv('/content/extraction_directory/twitter_training.csv')

In [4]:
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [5]:
df.columns

Index(['2401', 'Borderlands', 'Positive',
       'im getting on borderlands and i will murder you all ,'],
      dtype='object')

In [6]:
df['2401'].unique()

array([2401, 2402, 2403, ..., 9198, 9199, 9200])

In [7]:

"""
array(['Borderlands', 'CallOfDutyBlackopsColdWar', 'Amazon', 'Overwatch',
       'Xbox(Xseries)', 'NBA2K', 'Dota2', 'PlayStation5(PS5)',
       'WorldOfCraft', 'CS-GO', 'Google', 'AssassinsCreed', 'ApexLegends',
       'LeagueOfLegends', 'Fortnite', 'Microsoft', 'Hearthstone',
       'Battlefield', 'PlayerUnknownsBattlegrounds(PUBG)', 'Verizon',
       'HomeDepot', 'FIFA', 'RedDeadRedemption(RDR)', 'CallOfDuty',
       'TomClancysRainbowSix', 'Facebook', 'GrandTheftAuto(GTA)',
       'MaddenNFL', 'johnson&johnson', 'Cyberpunk2077',
       'TomClancysGhostRecon', 'Nvidia'], dtype=object)

"""
df['Borderlands'].unique()

array(['Borderlands', 'CallOfDutyBlackopsColdWar', 'Amazon', 'Overwatch',
       'Xbox(Xseries)', 'NBA2K', 'Dota2', 'PlayStation5(PS5)',
       'WorldOfCraft', 'CS-GO', 'Google', 'AssassinsCreed', 'ApexLegends',
       'LeagueOfLegends', 'Fortnite', 'Microsoft', 'Hearthstone',
       'Battlefield', 'PlayerUnknownsBattlegrounds(PUBG)', 'Verizon',
       'HomeDepot', 'FIFA', 'RedDeadRedemption(RDR)', 'CallOfDuty',
       'TomClancysRainbowSix', 'Facebook', 'GrandTheftAuto(GTA)',
       'MaddenNFL', 'johnson&johnson', 'Cyberpunk2077',
       'TomClancysGhostRecon', 'Nvidia'], dtype=object)

In [8]:
df.drop(columns=['2401', 'Borderlands'],inplace=True)

In [9]:
df.head()

,Positive,"im getting on borderlands and i will murder you all ,"
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [10]:
df.columns

Index(['Positive', 'im getting on borderlands and i will murder you all ,'], dtype='object')

In [11]:
df.Positive.unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [12]:
df.rename(columns={'Positive':'Target','im getting on borderlands and i will murder you all ,':'text'},inplace=True)

In [13]:
df.head()

,Target,text
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [14]:
df.Target.unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [15]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['Positive', 'Neutral', 'Negative', 'Irrelevant']])
df.Target = encoder.fit_transform(df[['Target']])

In [16]:
df.head()

,Target,text
0,0.0,I am coming to the borders and I will kill you...
1,0.0,im getting on borderlands and i will kill you ...
2,0.0,im coming on borderlands and i will murder you...
3,0.0,im getting on borderlands 2 and i will murder ...
4,0.0,im getting into borderlands and i can murder y...


In [17]:
df.Target.unique()

array([0., 1., 2., 3.])

In [18]:
df.head()

,Target,text
0,0.0,I am coming to the borders and I will kill you...
1,0.0,im getting on borderlands and i will kill you ...
2,0.0,im coming on borderlands and i will murder you...
3,0.0,im getting on borderlands 2 and i will murder ...
4,0.0,im getting into borderlands and i can murder y...


In [19]:
df.duplicated().sum()

np.int64(4909)

In [20]:
df.isnull().sum()

,0
Target,0
text,686


In [21]:
df.Target.value_counts()

,count
Target,
2.0,22542
0.0,20831
1.0,18318
3.0,12990


In [22]:
df.drop_duplicates(inplace=True)

In [23]:
df.dropna(inplace=True)

In [24]:
df.Target.value_counts()

,count
Target,
2.0,21237
0.0,19137
1.0,17110
3.0,12284


In [25]:
X = df.text

In [26]:
y = df.Target

In [27]:
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [28]:
text_lines = list()
for text in X:
  tokens = word_tokenize(text)
  tokens = [n.lower() for n in tokens]
  table  = str.maketrans('','',string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  words = [word for word in stripped if word.isalpha()]

  stop_words = set(stopwords.words('english'))
  words = [word for word in words if word not in stop_words]
  text_lines.append(words)

In [29]:
text_lines

[['coming', 'borders', 'kill'],
 ['im', 'getting', 'borderlands', 'kill'],
 ['im', 'coming', 'borderlands', 'murder'],
 ['im', 'getting', 'borderlands', 'murder'],
 ['im', 'getting', 'borderlands', 'murder'],
 ['spent',
  'hours',
  'making',
  'something',
  'fun',
  'nt',
  'know',
  'huge',
  'borderlands',
  'fan',
  'maya',
  'one',
  'favorite',
  'characters',
  'decided',
  'make',
  'wallpaper',
  'pc',
  'original',
  'image',
  'versus',
  'creation',
  'made',
  'enjoy'],
 ['spent',
  'couple',
  'hours',
  'something',
  'fun',
  'nt',
  'know',
  'huge',
  'borderlands',
  'fan',
  'maya',
  'one',
  'favorite',
  'characters',
  'decided',
  'make',
  'wallpaper',
  'pc',
  'original',
  'picture',
  'compared',
  'creation',
  'made',
  'fun',
  'pictwittercom'],
 ['spent',
  'hours',
  'something',
  'fun',
  'nt',
  'know',
  'huge',
  'borderlands',
  'fan',
  'maya',
  'one',
  'favorite',
  'characters'],
 ['spent',
  'hours',
  'making',
  'something',
  'fun',
  

In [30]:
len(text_lines)

69768

In [31]:
df.shape

(69768, 2)

In [32]:
%%capture
!pip install gensim

In [34]:
import gensim

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
EMBEDDING_DIM = 100
model = gensim.models.Word2Vec(sentences=text_lines,vector_size=EMBEDDING_DIM,window=5,min_count=1)

words = list(model.wv.index_to_key)
print(f'Vocabulary size: {len(words)}')


In [ ]:
model.wv.most_similar('university',topn=5)

In [ ]:
model.wv.get_vector('cut')

In [ ]:
print()

In [ ]:
filename = 'twitter_comments_embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)


In [ ]:
model.wv.most_similar(positive=['happy','smile'],negative='sad')

In [ ]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'])

In [ ]:
model.wv.similarity('men','happy')

In [ ]:
model.wv.similar_by_vector(10,50)

In [ ]:
embeddings_index = {}
f = open('/content/twitter_comments_embedding_word2vec.txt')
for line in f:
    print(line)
    values = line.split()
    word = values[0] # Since the word is at the 0th index
    coefs = np.asarray(values[1:]) # From the 1st position onwards the values are the embedding values
    embeddings_index[word] = coefs
f.close()


In [ ]:
!pip install keras_hub

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
VALIDATION_SPLIT = 0.2
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(text_lines)
sequences = tokenizer_obj.texts_to_sequences(text_lines) # Transforms text into a sequence of integers



In [ ]:
max_length = 100
word_index = tokenizer_obj.word_index
print(f'found {len(word_index)} unique tokens')
review_pad = pad_sequences(sequences,maxlen=max_length)
sentiment = df['Target'].values
print('Shape of review tensor: ',review_pad.shape)
print('Shape of sentiment tensor: ',sentiment.shape)

In [ ]:
tokenizer_obj.index_word

In [ ]:
tokenizer_obj.word_index

In [ ]:
review_pad[600]

In [ ]:
indices = np.arange(review_pad.shape[0])
print(indices)
np.random.shuffle(indices)
print(indices)

In [ ]:
review_pad = review_pad[indices]
sentiment = sentiment[indices]
num_validation_samples = int(VALIDATION_SPLIT * review_pad.shape[0])

In [ ]:
X_train_pad = review_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
X_test_pad = review_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

In [57]:
print('Shape of Training dataset X:', X_train_pad.shape)
print('Shape of Training dataset Y:', y_train.shape)
print('Shape of Validation dataset X:', X_test_pad.shape)
print('Shape of Valodation daatset Y:', y_test.shape)


Shape of Training dataset X: (55815, 100)
Shape of Training dataset Y: (55815,)
Shape of Validation dataset X: (13953, 100)
Shape of Valodation daatset Y: (13953,)


In [58]:
EMBEDDING_DIM =100
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
# words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [65]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU, Input
# from keras.layers import Embedding
from tensorflow.keras.initializers import Constant
# define model
# model = Sequential()
# embedding_layer = Embedding(num_words, EMBEDDING_DIM, embeddings_initializer=Constant(embedding_matrix), input_length=max_length, trainable=False)
# model.add(embedding_layer)
# model.add(LSTM(units=32,  dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(1, activation='sigmoid'))


inputs = Input(shape=(max_length,))
embedding_layer = Embedding(num_words, EMBEDDING_DIM, embeddings_initializer=Constant(embedding_matrix), trainable=False)(inputs)
lstm_layer = LSTM(units=32,dropout=0.2, recurrent_dropout=0.2)(embedding_layer)
outputs = Dense(1,activation='sigmoid')(lstm_layer)
model = Model(inputs=inputs,outputs=outputs)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print('Summary of the built model...')
model.summary()



Summary of the built model...


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_4 (Embedding)         │ (None, 100, 100)       │     3,055,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 32)             │        17,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,072,957 (11.72 MB)

 Trainable params: 17,057 (66.63 KB)

 Non-trainable params: 3,055,900 (11.66 MB)

In [ ]:
print('Train...')
model.fit(X_train_pad, y_train, batch_size=128, epochs=25, validation_data=(X_test_pad, y_test), verbose=2)

Train...
Epoch 1/25
437/437 - 129s - 295ms/step - accuracy: 0.2447 - loss: -4.8739e+00 - val_accuracy: 0.2468 - val_loss: -7.8322e+00
Epoch 2/25
437/437 - 123s - 281ms/step - accuracy: 0.2449 - loss: -1.0730e+01 - val_accuracy: 0.2468 - val_loss: -1.3232e+01
Epoch 3/25
437/437 - 143s - 326ms/step - accuracy: 0.2449 - loss: -1.6187e+01 - val_accuracy: 0.2468 - val_loss: -1.8562e+01
Epoch 4/25
437/437 - 123s - 281ms/step - accuracy: 0.2449 - loss: -2.1599e+01 - val_accuracy: 0.2468 - val_loss: -2.3850e+01
Epoch 5/25
437/437 - 123s - 281ms/step - accuracy: 0.2449 - loss: -2.7011e+01 - val_accuracy: 0.2468 - val_loss: -2.9143e+01
Epoch 6/25
437/437 - 123s - 282ms/step - accuracy: 0.2449 - loss: -3.2398e+01 - val_accuracy: 0.2468 - val_loss: -3.4417e+01
Epoch 7/25
437/437 - 123s - 281ms/step - accuracy: 0.2449 - loss: -3.7772e+01 - val_accuracy: 0.2468 - val_loss: -3.9695e+01
Epoch 8/25
437/437 - 123s - 282ms/step - accuracy: 0.2449 - loss: -4.3146e+01 - val_accuracy: 0.2468 - val_loss: -4.